# Setting Up All Artifacts details

In [1]:
import os
os.environ['QNN_SDK_ROOT']="/local/mnt/workspace/gokul/QNN/2.19.0.240124" #set up your qnn path here.
os.environ['ANDROID_NDK_ROOT']="/local/mnt/workspace/gokul/android-ndk-r25c"
os.environ['FOLDER_WITH_ARTIFACTS']="SCI"
os.environ['RAW_FILE_FOLDER']="raw"
os.environ['ONDEVICE_FOLDER']="SCI"
os.environ['MODEL_NAME']="sci_difficult_fp32"
os.environ['QUANTIZED_MODEL_NAME']="sci_difficult_w8a8"
os.environ['MODEL_PATH']="models/sci_difficult_fp32"
os.environ['QUANTIZED_MODEL_PATH']="models/sci_difficult_w8a8"
os.environ['TARGET_INPUT_LIST']="list.txt"
os.environ['DEVICE_HOST']="localhost"
os.environ['DEVICE_ID']="bc468c1d" #fill your device-id. Use command "adb devices" to get devices names. example :"e18d5d0"
os.environ['QNN_TARGET_ARCH']="aarch64-android"
os.environ['QNN_TARGET_STL']="libQnnHtp.so"

In [2]:
from PIL import Image
import glob
import os
import cv2
import numpy as np
import torch
import os
import shutil

# Getting The Model

- **If You Already have the models in the dlc folder no need to run this cell**
- [ https://github.com/vis-opt-group/SCI.git ](Link of the Actual Model)
- Self-Calibrated Illumination (SCI) Learning 

In [3]:
%%bash
git clone https://github.com/vis-opt-group/SCI
#git reset  --hard 808e70644191a63c936bd4ce73ce3f10fbc02ec8
cp SCI.patch SCI

fatal: destination path 'SCI' already exists and is not an empty directory.


In [4]:
command1="cd SCI;patch -i SCI.patch"
os.system(command1)

patching file model.py
Reversed (or previously applied) patch detected!  Assume -R? [n] 
Apply anyway? [n] 
Skipping patch.
1 out of 1 hunk ignored -- saving rejects to file model.py.rej


256

In [5]:
%%bash
cp generate_model.py SCI/
cd SCI
python generate_model.py

ONNX Model Saved Successfully


#### Getting the Non Quantized DLC File

#### Understanding the Architecture of the Model

## Getting Dataset

Use the dataset of your choice to validate the pre-processing, and post processing steps given in this notebook
  

## Preprocessing the data to generate raw file

In [6]:
def preprocess(ll_img, hl_img):
    ll_img = np.array(ll_img, dtype='float32')
    hl_img = np.array(hl_img, dtype='float32')
    
    ll_img = np.uint8(np.clip(ll_img, 0., 255.))  # this is to simulate matlab's imwrite operation
    hl_img = np.uint8(np.clip(hl_img, 0., 255.))  # this is to simulate matlab's imwrite operation
    
    ll_img = np.uint8(ll_img)
    hl_img = np.uint8(hl_img)
    
    ll_img = convert_image(ll_img, source='array', target='[0, 1]')
    hl_img = convert_image(hl_img, source='array', target='[0, 1]')
    
    return ll_img, hl_img

In [7]:
def convert_image(img, source, target):
    if source == 'array':
        img = torch.from_numpy(img.transpose((2, 0, 1))).contiguous()#chw
        img = img.to(dtype=torch.float32).div(255) 
    elif source == '[0, 1]':
        img = torch.clamp(img, 0, 1)  # useful to post-process output of models that can overspill
    
    if target == '[0, 1]':
        pass  # already in [0, 1]
    return img

In [8]:
def post_process(img):
    img = img.detach().cpu().numpy()
    img = np.clip(255. * img, 0., 255.)
    img = np.uint8(img)
    img = img.transpose(1, 2, 0)#hwc
    return img

In [9]:
def load_dataset(test_images_dir):
    # Input images for the model
    INPUTS_LL = []
    # Post-processed images for visualization
    IMAGES_LL = [] # LL:Low Light
    IMAGES_HL = [] # HL:High Light

    # Load the test images
    count=0
    for img_path in glob.glob(os.path.join(test_images_dir, '*')):
        l_img = cv2.resize(cv2.imread(img_path),[480,640],interpolation=cv2.INTER_CUBIC)
        l_img = cv2.cvtColor(l_img, cv2.COLOR_BGR2RGB)
        
        h_img = cv2.resize(cv2.imread(img_path.replace("low","high")),[480,640],interpolation=cv2.INTER_CUBIC)
        h_img = cv2.cvtColor(h_img, cv2.COLOR_BGR2RGB)
        
        ll_img, hl_img = preprocess(l_img, h_img)#chw

        INPUTS_LL.append(ll_img)#chw
        IMAGES_LL.append(post_process(ll_img))#hwc
        IMAGES_HL.append(post_process(hl_img))#hwc
    return INPUTS_LL, IMAGES_LL, IMAGES_HL

**Converting the low dataset to raw file format to give it to the model and do the inference**

In [10]:
os.makedirs('raw', exist_ok=True)

In [11]:
test_images_dir = "eval15/low"
INPUTS_LL, IMAGES_LL, IMAGES_HL = load_dataset(test_images_dir)
print(len(INPUTS_LL),len(IMAGES_LL),len(IMAGES_HL))
for i, img_ll in enumerate(INPUTS_LL):
    img_ll = img_ll.cpu().detach().numpy()
    img_ll = img_ll.astype(np.float32)
    fid = open("raw/img_"+str(i)+ ".raw", 'wb')
    img_ll.tofile(fid)
    fid.close()

0 0 0


In [12]:
total_iter = 15
print("Generating input_list \"small_raw_list.txt\" with {} iterations".format(total_iter))
with open("list.txt",'w') as f:
    for i in range(total_iter):
        f.write("raw/img_{}.raw\n".format(i))

Generating input_list "small_raw_list.txt" with 15 iterations


#### Getting the Quantized Model

In [13]:
%%bash
source ${QNN_SDK_ROOT}/bin/envsetup.sh
export PATH=${ANDROID_NDK_ROOT}:${PATH}
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-onnx-converter --input_network sci_difficult.onnx --output_path ${MODEL_PATH}.cpp
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-model-lib-generator -c ${MODEL_PATH}.cpp -b ${MODEL_PATH}.bin -o models/model_libs
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-context-binary-generator \
              --backend ${QNN_SDK_ROOT}/lib/x86_64-linux-clang/libQnnHtp.so \
              --model models/model_libs/x86_64-linux-clang/lib${MODEL_NAME}.so \
              --binary_file ${MODEL_NAME}.serialized

In [14]:
%%bash
source ${QNN_SDK_ROOT}/bin/envsetup.sh
export PATH=${ANDROID_NDK_ROOT}:${PATH}
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-onnx-converter --input_network sci_difficult.onnx --output_path ${QUANTIZED_MODEL_PATH}.cpp --input_list list.txt \
                --param_quantizer "adjusted" --act_quantizer "enhanced"
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-model-lib-generator -c ${QUANTIZED_MODEL_PATH}.cpp -b ${QUANTIZED_MODEL_PATH}.bin -o models/model_libs2
${QNN_SDK_ROOT}/bin/x86_64-linux-clang/qnn-context-binary-generator \
              --backend ${QNN_SDK_ROOT}/lib/x86_64-linux-clang/libQnnHtp.so \
              --model models/model_libs2/x86_64-linux-clang/lib${QUANTIZED_MODEL_NAME}.so \
              --binary_file ${QUANTIZED_MODEL_NAME}.serialized

[INFO] AISW SDK environment set
[INFO] QNN_SDK_ROOT: /local/mnt/workspace/gokul/QNN/2.19.0.240124


2024-03-04 09:42:36,750 - 235 - INFO - Simplified model validation is successful
2024-03-04 09:42:37,578 - 235 - INFO - Saving QNN Model...


IrQuantizer: Quantizer param type: adjusted will be deprecated in future releases
IrQuantizer: Quantizer type: adjusted is no longer supported. Using TF quantizer instead


2024-03-04 09:42:38,817 - 235 - INFO - Model CPP saved at: models/sci_difficult_w8a8.cpp 
2024-03-04 09:42:38,817 - 235 - INFO - Model BIN saved at: /local/mnt/workspace/gokul/models-for-solutions/02-low-light-enhancement/SCI/models/sci_difficult_w8a8.bin 
2024-03-04 09:42:38,817 - 235 - INFO - Conversion complete!


     0.1ms [  INFO ] Inferences will run in sync mode
     0.1ms [  INFO ] Initializing logging in the backend. Callback: [0x7fd9ef440e10], Log Level: [3]
     0.1ms [  INFO ] No BackendExtensions lib provided;initializing NetRunBackend Interface
     0.1ms [WARNING] Unable to find a device with NetRunDeviceKeyDefault in Library NetRunBackendLibKeyDefault
     0.2ms [  INFO ] Entering QuantizeRuntimeApp flow
     0.4ms [  INFO ] CpuGraph::finalize
    32.1ms [  INFO ] CpuGraph::execute
   122.8ms [  INFO ] cleaning up resources for input tensors
   122.9ms [  INFO ] cleaning up resources for output tensors
   143.9ms [  INFO ] CpuGraph::execute
   209.8ms [  INFO ] cleaning up resources for input tensors
   209.8ms [  INFO ] cleaning up resources for output tensors
   223.0ms [  INFO ] CpuGraph::execute
   286.8ms [  INFO ] cleaning up resources for input tensors
   286.8ms [  INFO ] cleaning up resources for output tensors
   299.1ms [  INFO ] CpuGraph::execute
   359.2ms [  INFO ] cl

jni/sci_difficult_w8a8.cpp:39:47: warning: mixture of designated and non-designated initializers in the same initializer list is a C99 extension [-Wc99-designator]
                                              .id=0,
                                              ^~~~~
jni/QnnWrapperUtils.hpp:77:17: note: expanded from macro 'VALIDATE'
    retStatus = value;                                                                           \
                ^~~~~
jni/sci_difficult_w8a8.cpp:50:47: note: first non-designated initializer is here
                                              {.clientBuf= { .data=nullptr,
                                              ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~
jni/QnnWrapperUtils.hpp:77:17: note: expanded from macro 'VALIDATE'
    retStatus = value;                                                                           \
                ^~~~~
jni/sci_difficult_w8a8.cpp:37:45: warning: mixture of designated and non-designated initializers in the same initializer

/local/mnt/workspace/gokul/android-ndk-r25c/toolchains/llvm/prebuilt/linux-x86_64/bin/llvm-objcopy -I binary -O elf64-littleaarch64 -B aarch64 obj/binary/enhance_in_conv_0_bias.raw obj/local/arm64-v8a/objs/sci_difficult_w8a8//enhance_in_conv_0_bias.o
/local/mnt/workspace/gokul/android-ndk-r25c/toolchains/llvm/prebuilt/linux-x86_64/bin/llvm-objcopy -I binary -O elf64-littleaarch64 -B aarch64 obj/binary/enhance_in_conv_0_weight.raw obj/local/arm64-v8a/objs/sci_difficult_w8a8//enhance_in_conv_0_weight.o
/local/mnt/workspace/gokul/android-ndk-r25c/toolchains/llvm/prebuilt/linux-x86_64/bin/llvm-objcopy -I binary -O elf64-littleaarch64 -B aarch64 obj/binary/enhance_out_conv_0_bias.raw obj/local/arm64-v8a/objs/sci_difficult_w8a8//enhance_out_conv_0_bias.o
/local/mnt/workspace/gokul/android-ndk-r25c/toolchains/llvm/prebuilt/linux-x86_64/bin/llvm-objcopy -I binary -O elf64-littleaarch64 -B aarch64 obj/binary/enhance_out_conv_0_weight.raw obj/local/arm64-v8a/objs/sci_difficult_w8a8//enhance_out_

## Inferencing the Model

In [15]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/qnnexample/$QNN_TARGET_ARCH/bin" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/qnnexample/$QNN_TARGET_ARCH/lib" && $DEVICE_SHELL shell "mkdir -p /data/local/tmp/qnnexample/dsp/lib"

In [16]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell "mkdir -p /data/local/tmp/$ONDEVICE_FOLDER"

In [17]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push $QNN_SDK_ROOT/lib/$QNN_TARGET_ARCH/$QNN_TARGET_STL /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $QNN_SDK_ROOT/bin/$QNN_TARGET_ARCH/qnn-net-run /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $QNN_SDK_ROOT/lib/hexagon-v73/unsigned/*.so /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $QNN_SDK_ROOT/lib/$QNN_TARGET_ARCH/*.so /data/local/tmp/$ONDEVICE_FOLDER

/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/aarch64-android/libQnnHtp.so: 1 file pushed, 0 skipped. 400.5 MB/s (1545520 bytes in 0.004s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/bin/aarch64-android/qnn-net-run: 1 file pushed, 0 skipped. 424.4 MB/s (1441208 bytes in 0.003s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/hexagon-v73/unsigned/libCalculator_skel.so: 1 file pushed, 0 skipped. 223.1 MB/s (7128 bytes in 0.000s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/hexagon-v73/unsigned/libQnnHtpV73QemuDriver.so: 1 file pushed, 0 skipped. 501.2 MB/s (7601584 bytes in 0.014s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/hexagon-v73/unsigned/libQnnHtpV73Skel.so: 1 file pushed, 0 skipped. 524.3 MB/s (7360784 bytes in 0.013s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/hexagon-v73/unsigned/libQnnHtpV73.so: 1 file pushed, 0 skipped. 521.0 MB/s (8025008 bytes in 0.015s)
/local/mnt/workspace/gokul/QNN/2.19.0.240124/lib/hexagon-v73/unsigned/libQnnSaver.so: 1 file pushed,

In [18]:
%%bash
#find ./raw -name *.raw > list.txt
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL push output/${QUANTIZED_MODEL_NAME}.serialized.bin /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push output/${MODEL_NAME}.serialized.bin /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push models/model_libs2/aarch64-android/lib${QUANTIZED_MODEL_NAME}.so /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $RAW_FILE_FOLDER /data/local/tmp/$ONDEVICE_FOLDER
$DEVICE_SHELL push $TARGET_INPUT_LIST /data/local/tmp/$ONDEVICE_FOLDER

output/sci_difficult_w8a8.serialized.bin: 1 file pushed, 0 skipped. 205.8 MB/s (95896 bytes in 0.000s)
output/sci_difficult_fp32.serialized.bin: 1 file pushed, 0 skipped. 172.0 MB/s (108304 bytes in 0.001s)
models/model_libs2/aarch64-android/libsci_difficult_w8a8.so: 1 file pushed, 0 skipped. 313.0 MB/s (326408 bytes in 0.001s)
raw/: 15 files pushed, 0 skipped. 317.7 MB/s (55296000 bytes in 0.166s)
list.txt: 1 file pushed, 0 skipped. 5.7 MB/s (215 bytes in 0.000s)


#### Inferencing the Quantized Model

In [19]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export ONDEVICE_FOLDER="SCI"
export device_path=/data/local/tmp/$ONDEVICE_FOLDER
export LD_LIBRARY_PATH=$device_path
export ADSP_LIBRARY_PATH="$device_path"
export PATH=$PATH:$device_path
export QUANTIZED_MODEL_NAME="sci_difficult_w8a8"
cd $device_path
export OUTPUT_FOLDER=OUTPUT_Quant_DSP
cd /data/local/tmp/$ONDEVICE_FOLDER  && 
./qnn-net-run --backend libQnnHtp.so --input_list list.txt --retrieve_context ${QUANTIZED_MODEL_NAME}.serialized.bin --output_dir $OUTPUT_FOLDER

qnn-net-run pid:13015


#### Inferencing the 32b Model

In [20]:
%%bash
export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL shell
export ONDEVICE_FOLDER="SCI"
export device_path=/data/local/tmp/$ONDEVICE_FOLDER
export LD_LIBRARY_PATH=$device_path
export ADSP_LIBRARY_PATH="$device_path"
export PATH=$PATH:$device_path
export QUANTIZED_MODEL_NAME="sci_difficult_w8a8"
cd $device_path
export OUTPUT_FOLDER=OUTPUT_CPU
cd /data/local/tmp/$ONDEVICE_FOLDER  && 
./qnn-net-run --backend libQnnCpu.so --input_list list.txt --model lib${QUANTIZED_MODEL_NAME}.so --output_dir $OUTPUT_FOLDER

qnn-net-run pid:13023


#### Pulling the Output

In [21]:
%%bash
rm -rf OUTPUT_Quant_DSP
rm -rf OUTPUT_CPU

In [22]:
%%bash

export DEVICE_SHELL="adb -H $DEVICE_HOST -s $DEVICE_ID"
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_Quant_DSP output/OUTPUT_Quant_DSP
$DEVICE_SHELL pull /data/local/tmp/$ONDEVICE_FOLDER/OUTPUT_CPU output/OUTPUT_CPU

/data/local/tmp/SCI/OUTPUT_Quant_DSP/: 30 files pulled, 0 skipped. 152.4 MB/s (110592000 bytes in 0.692s)
/data/local/tmp/SCI/OUTPUT_CPU/: 30 files pulled, 0 skipped. 165.3 MB/s (110592000 bytes in 0.638s)


### Post Processing and Calculating PSNR Value

In [23]:
import math
def compute_psnr(img_pred, img_true, data_range=255., eps=1e-8):
    err = (img_pred - img_true) ** 2 
    err = np.mean(err)
    return 10. * math.log10((data_range ** 2) / (err + eps))
def evaluate_average_psnr(el_images, hl_images): #(enhanced_light<model o/p>, high_light <groud_truth>)
    psnr = []
    for i in range(len(el_images)):
        el_img = cv2.imread(el_images[i], 1)
        hl_img = cv2.imread(hl_images[i], 1)
        psnr.append(compute_psnr(el_img,hl_img))
    average_psnr = np.mean(np.array(psnr))
    return average_psnr

In [24]:
def post_process_enhanced(img):
    img = np.fromfile(img, np.float32)
    img = img.reshape((3,640,480)).astype(np.float32)
    img = np.clip(img * 255. , 0., 255.)
    img = np.uint8(img)
    img = img.transpose(1, 2, 0)#hwc
    return img